In [1]:
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from torch.nn.functional import one_hot

In [2]:
with open('names.txt') as f:
    data = f.read().split('\n')
data[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [3]:
characters = list(set([i for i in ''.join(data)]))
characters.insert(1, '.')
hmap = {}
for n in range(0,27):
    hmap[sorted(characters)[n]] = n
print(hmap)

{'.': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


In [4]:
# Create dataset
X, y = [],[]

for name in data:
    name = '.' + name + '.'
    for ch1, ch2 in zip(name, name[1:]):
        X.append(hmap[ch1])
        y.append(hmap[ch2])
    break

([0, 5, 13, 13, 1], [5, 13, 13, 1, 0])

In [7]:
X = torch.tensor(X)
y = torch.tensor(y)
X.shape, y.shape

/tmp/ipykernel_8910/3222465247.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X)
/tmp/ipykernel_8910/3222465247.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y)


(torch.Size([5]), torch.Size([5]))